In [ ]:
import sys
import os

import numpy
import scipy
from matplotlib import pyplot as plt
from scipy import stats as stats

from red_utils import find_nearest, cplot

In [ ]:
%run red_utils

In [ ]:
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

In [ ]:
%run red_likelihood.py

In [ ]:
pol = 'ee'
freq_channel = 300
time_integration = 30
bad_ants = [0, 2, 11, 24, 50, 53, 54, 67, 69, 98, 122, 136, 139]

In [ ]:
mdm_fn = '/Users/matyasmolnar/Downloads/HERA_Data/robust_cal/zen.2458098.43869.HH.uvh5'
if os.path.exists(mdm_fn):
    filename = mdm_fn
else:
    filename = './zen.2458098.43869.HH.uvh5'

In [ ]:
hdraw, cRedG, cData = group_data(filename, pol, freq_channel, bad_ants)

In [ ]:
def plot_red_vis(data, redg, vis_type='amp'):
    """Pot visibility amplitudes or phases, grouped by redundant type"""
    vis_calc = {'amp':np.absolute, 'phase': np.angle}
    bl_id_seperations = numpy.unique(redg[:, 0], return_index=True)[1][1:]
    fig, ax = plt.subplots(figsize=(13, 4))
    ax.matshow(vis_calc[vis_type](data), aspect='auto')
    for bl_id_seperation in bl_id_seperations:
        plt.axvline(x=bl_id_seperation, color='white', linestyle='-.', linewidth=1)
    ax.grid(False)
    ax.set_xlabel('Baseline ID')
    ax.set_ylabel('Time Integration')
    plt.show()

In [ ]:
plot_red_vis(cData, cRedG, vis_type='amp')

# Redundant calibration

Fundamentally, the problem of calibration boils down to the measurement equation:

$$ V_{ij}^{\text{obs}} (\nu) = g_i (\nu) g_j^* (\nu) V_{ij}^{\text{true}}(\nu) + n_{ij} (\nu) $$

where the observed visibility $V_{ij}$ between antennas $i$ and $j$ at a given time and frequency is related to the true, underlying visibility by a pair of complex and frequency-dependent gain factors, $g_i$ and $g_j$, if we assume per-antenna gains, along with uncorrelated Gaussian random noise $n_{ij}$. The ultimate aim of calibration is to solve for these gains and visibilities. 

For a redundant array, we have a system of equations for all antenna pairs $i$ and $j$, given by

$$ V_{ij}^{\text{obs}} (\nu) = g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) $$

where $U_{\alpha}(\nu) = V(\mathbf{r}_i-\mathbf{r}_j)$, the visibility for the baseline vector $ \mathbf{b}_{ij} = \mathbf{r}_i-\mathbf{r}_j$, which corresponds to a redundant baseline set that we index by $\alpha$. Two pairs of identical elements with precisely the same baseline separation between them are sensitive to the same mode on the sky, and are thus measuring the same $V_{ij}^{\text{true}}$.

In highly redundant arrays, there are many more observations than there are unique baselines. For the full HERA array, there are 331 elements in the hexagonal core, corresponding to $N_{\mathrm{bl}} = 331(331-1)/2 = 54,615$ baselines. The hexagonal core only has 630 unique baseline separations, this means we have a non-linear system of $54,615$ equations to determine 630 unique visibilities and the 331 complex gains; the system is vastly overdetermined.

## Relative calibration

### Gaussian distribution

By imposing that the true sky visibilities from redundant baselines are equal, relative redundant calibration solves for the true sky visibilities for each redundant baseline set $V_{i-j}^{\text{sol}}$, alongside the gains of the individual antennas.

Assuming Gaussian uncorrelated noise with variance $\sigma_{ij}^2$, a maximum-likelihood estimate for the gains and sky visibilities can be constructed:

$$ \mathcal{L} (\{g_i(\nu)\}, \{U_{\alpha}(\nu)\} | \{V_{ij}^{\text{obs}}(\nu)\} \propto \prod_{\nu} \prod_{\alpha} \prod_{\{i,j\}_{\alpha}} \exp{\left( -\frac{1}{2} \left( \frac{ \left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^{*} (\nu) U_{\alpha}(\nu) \right|^2}{\sigma_{ij}^2(\nu)} \right) \right)} $$

where $\{i,j\}_{\alpha}$ are sets of antennas that belong to each redundant baseline type $\alpha$. Maximizing this function is equivalent to minimizing:

$$ \chi_{\mathrm{rel}}^{2} (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|^2 }{\sigma_{ij}^2(\nu)} $$

by varying $g_k(\nu)$ and $U_{\alpha}(\nu)$ for each frequency $\nu$.

This non-linear least-squares optimization can be done independently between frequencies and time.

### Cauchy distribution

We no longer assume that the noise in the measurement equation is Gaussian. Empirically, it is found that visibility observations can have a lot of outliers, and the visibilities follow a distribution with fatter tails than a Gaussian. We wish to be insensitive to outliers, so we need to employ robust statistics to adequately deal with such measurements.

Instead of fitting the visibilities to a Gaussian, we fit them to a Cauchy distribution, when solving the redundant calibration parameter estimation.

The Cauchy distribution is given by

$$ f(x; x_0, \gamma) = \frac{1}{\pi \gamma \left[ 1 + \left( \frac{x - x_0}{\gamma}  \right)^2  \right] }$$

where $x_0$ is the location parameter (the median) and $\gamma$ is the scale parameter, which specifies the half-width at half-maximum (HWHM).

Working in a maximum likelihood framework, the likelihood when solving for the measurement equation for redundant baseline sets is given by

$$ \mathcal{L} = \prod_{\nu} \prod_{\alpha} \prod_{\{i,j\}_{\alpha}} \frac{1}{\pi \gamma_{\alpha} (\nu)} \left[ 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|}{\gamma_{\alpha} (\nu)}  \right)^2  \right]^{-1} $$

The negative log-likelihood is therefore given by

$$ -\ln(\mathcal{L}) (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha} (\nu)) + \ln \left( 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|}{\gamma_{\alpha}(\nu)}  \right)^2 \right) $$

where we have dropped the sum over frequency, as we can solve independently for each frequency.

In [ ]:
# Finding all the antennas used in our flagged data
ants = numpy.unique(cRedG[:,1:])
no_unq_bls = numpy.unique(cRedG[:, 0]).size

In [ ]:
# Setup initial parameters
xvis = numpy.ones(no_unq_bls*2) # Number of unique baselines; complex vis
xgains = numpy.ones(ants.size*2) # Complex gain
rel_xparams = numpy.hstack([xvis, xgains])

In [ ]:
# This creates a compiled version of likelihood where the first (the redundant groups),
# second (the error distribution) and third (observed visibilities) parameters 
# have already been filled in [-- this is "partial" fn application]
ff = jit(functools.partial(relative_logLkl, relabelAnts(cRedG), 'cauchy', \
                           cData[time_integration, :]))
res_rel = scipy.optimize.minimize(ff, rel_xparams, jac=jacrev(ff))
print(res_rel['message'])

In [ ]:
def split_rel_results(results):
    """Split results from minimization into visibility and gains arrays"""
    vis_params, gains_params = numpy.split(results['x'], [no_unq_bls*2,])
    res_vis = makeCArray(vis_params)
    res_gains = makeCArray(gains_params)
    return res_vis, res_gains

In [ ]:
res_rel_vis, res_rel_gains = split_rel_results(res_rel)

In [ ]:
# plotting residual for calculated visibilities
rel_residual = cData[time_integration, :] - gVis(res_rel_vis, cRedG, res_rel_gains)
cplot(rel_residual, xlabel='Baseline', ylabel='Residual')

## Optimal calibration (solving for degeneracies)

Relative calibration yields degenerate solutions, that can be parameterized as four terms per frequency:
 - Overall amplitude $A(\nu)$
 - Overall phase $\Delta(\nu)$
 - Phase gradient components $\Delta_x(\nu)$ and $\Delta_y(\nu)$
 
The below transformations of these degenerate parameters leave $-\ln(\mathcal{L})$ (for both Gaussian and Cauchy distributions) unchanged:
 - $g_i \rightarrow A g_i$ accompanied by $U_{\alpha} \rightarrow A^{-2} U_{\alpha}$
 - $g_k = |g_k|e^{i\phi_k} \rightarrow |g_k|e^{i(\phi_k + \Delta)}$ corresponds to $g_k g_l^{*} = |g_k| |g_l| e^{i(\phi_k - \phi_l)} \rightarrow |g_k| |g_l| e^{i(\phi_k + \Delta - \phi_l - \Delta)} = g_k g_l^{*}$
 - $g_k = |g_k|e^{i\phi_k} \rightarrow |g_k|e^{i(\phi_k + \Delta_x x_k + \Delta_y y_k)}$ accompanied by $U_{\alpha} = |U_{\alpha}| e^{i\phi_{\alpha}} \rightarrow |U_{\alpha}| e^{i(\phi_{\alpha} - \Delta_x x_{\alpha} - \Delta_y y_{\alpha})}$
 
Where in the last line, the array is assumed to be co-planar, and $(x_k, y_k)$ are the coordinates of the position of antennas $k$, and $(x_{\alpha}, y_{\alpha})$ are the separations of the antennas that form baselines in redundant set $\alpha$.

These degeneracies must be constrained - this is the "absolute" part of redundant calibration. The degenerate parameters can either calculated from a sky model, or can be solved for using optimal absolute calibration, which calculates the degenerate parameters directly from the $\chi^2$ or $-\ln(\mathcal{L})$ by applying a few conditions.

 - Gaussian distribution:
 
$$ \chi_{\text{opt}}^2 (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left|  V_{ij}^{\text{obs}} (\nu) - h_i (\nu) h_j^{*} (\nu) W_{\alpha} (\nu) \right|^2 }{\sigma_{ij}^2(\nu)} $$


where

$$ W_{\alpha} (\nu) = A^2(\nu) e^{i \left[ \Delta_{x} (\nu) x_{\alpha} + \Delta_{y} (\nu) y_{\alpha} \right]} U_{\alpha} $$

In [ ]:
# Setup initial parameters
xamp = 1.
xoverall_phase = 0.
xphase_grad_x = 0.
xphase_grad_y = 0.
opt_xparams = numpy.hstack([xgains, xamp, xoverall_phase, xphase_grad_x, xphase_grad_y])

In [ ]:
ref_ant = 12 # reference antenna number to set overall phase

constraints = Opt_Constraints(ants, ref_ant, hdraw.antpos)
cons = [{'type': 'eq', 'fun': constraints.avg_amp},
        {'type': 'eq', 'fun': constraints.avg_phase},
        {'type': 'eq', 'fun': constraints.ref_phase},
        {'type': 'eq', 'fun': constraints.phase_grad}]

In [ ]:
ff = jit(functools.partial(optimal_logLkl, relabelAnts(cRedG), 'gaussian', \
                           red_ant_sep(cRedG, hdraw.antpos), cData[time_integration, :], \
                           res_rel_vis))
res_opt = scipy.optimize.minimize(ff, opt_xparams, constraints=cons, jac=jacrev(ff), \
                                  method='trust-constr')
print(res_opt['message'])

In [ ]:
new_gain_params, new_deg_params = np.split(res_opt['x'], [ants.size*2,])
new_gains = makeCArray(new_gain_params)

In [ ]:
print('Degenerate parameters: {}'.format(str(new_deg_params)[1: -1]))
print('Amplitude average: {}'.format(np.average(np.abs(new_gains))))
print('Phase circular mean: {}'.format(stats.circmean(np.angle(new_gains))))

# Comparing relative calibrations

To monitor the consistency of the "true" visibility solutions for each baseline set, we calculate the relative calibration visibility solutions from another JD and compare them to the relative solutions of the initial dataset. These should be consistent, up to the degenerate parameters $A$, $\Delta$, $\Delta_x$ and $\Delta_y$ - solving for these parameters with an MLE framework enables us to compare these solutions.

We need to minimize:

- Assuming Gaussian distributions:

$$ \chi^2_{\text{deg}} (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| U_{\alpha}' (\nu) - W_{\alpha} (\nu) \right|^2 }{\sigma_{\alpha}^2(\nu)} $$

- Assuming Cauchy distributions:

$$-\ln(\mathcal{L}) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha}) + \ln \left( 1 + \left( \frac{\left| U_{\alpha}' (\nu) - W_{\alpha} (\nu) \right|}{\gamma_{\alpha}}  \right)^2 \right) $$

In [ ]:
mdm_fn2 = '/Users/matyasmolnar/Downloads/HERA_Data/robust_cal/zen.2458099.43869.HH.uvh5'
if os.path.exists(mdm_fn2):
    filename2 = mdm_fn2
else:
    filename2 = './zen.2458099.43869.HH.uvh5'

In [ ]:
hdraw2, cRedG2, cData2 = group_data(filename2, pol, freq_channel, bad_ants)

In [ ]:
# Find time integration in dataset 2 that corresponds to closest LST to that of dataset 1
# This currently assumes that dataset 2 contains the correct time integration...
time_integration2 = find_nearest(hdraw2.lsts, hdraw.lsts[time_integration])[1]

In [ ]:
print('Do {} and {} have the same redundant grouping? {}'.format(os.path.basename(filename), \
      os.path.basename(filename2), (cRedG==cRedG2).all()))

In [ ]:
plot_red_vis(cData2, cRedG2, vis_type='amp')

In [ ]:
# Relative calibration for the 2nd dataset
ff = jit(functools.partial(relative_logLkl, relabelAnts(cRedG2), 'cauchy', \
                           cData2[time_integration2, :]))
res_rel2 = scipy.optimize.minimize(ff, rel_xparams, jac=jacrev(ff))
print(res_rel2['message'])

In [ ]:
res_rel_vis2, res_rel_gains2 = split_rel_results(res_rel2)

In [ ]:
deg_xparams = numpy.hstack([xamp, xoverall_phase, xphase_grad_x, xphase_grad_y])
cons2 = [cons[-1]] # only select phase gradient constraint

In [ ]:
#F Calculate degenerate parameters that translate between dataset 1 and dataset 2
ff = jit(functools.partial(deg_logLkl, 'cauchy', red_ant_sep(cRedG2, hdraw2.antpos), \
                           res_rel_vis, res_rel_vis2))
res_deg = scipy.optimize.minimize(ff, deg_xparams, constraints=cons2, jac=jacrev(ff), \
                                  method='trust-constr')
print(res_deg['message'])

In [ ]:
res_deg['x']